# Chapter 3: Assigning Roles (Role Prompting)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
from mlx_lm import load, stream_generate

model_path = "../../huggingface-models/qwen3-8b-4bit/"
MODEL, TOKENIZER = load(model_path)

In [2]:
from IPython.display import Markdown, display

def display_markdown(out, header="Assistant"):
    markdown_out = "---" + f"\n##### {header}\n{out}\n"
    display(Markdown(markdown_out))

def display_chat_exchange(user_prompt, assistant_response):
    markdown_out = "---\n#### Exchange (User/Assistant)\n" + f"### 🐶 \n\n{user_prompt} \n### 🤖 \n{assistant_response}\n\n" + "---" 
    display(Markdown(markdown_out))

In [3]:
def get_completion(
        prompt: str, 
        system_prompt=""):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    tokenized_prompt = TOKENIZER.apply_chat_template(
        messages,
        add_generation_prompt=True,
        enable_thinking=False
    )

    return TOKENIZER.decode(tokenized_prompt),  "".join([
        response.text
        for response in stream_generate(
            MODEL, 
            TOKENIZER, 
            tokenized_prompt, 
            max_tokens=-1 
        )
    ])

In [ ]:
system_prompt = ""
user_prompt = "Who is the best basketball player of all time?"
raw_prompt, assistant_response = get_completion(
    user_prompt, 
    system_prompt=system_prompt
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange(user_prompt, assistant_response)

---

## Lesson

Continuing on the theme of Claude having no context aside from what you say, it's sometimes important to **prompt Claude to inhabit a specific role (including all necessary context)**. This is also known as role prompting. The more detail to the role context, the better.

**Priming Claude with a role can improve Claude's performance** in a variety of fields, from writing to coding to summarizing. It's like how humans can sometimes be helped when told to "think like a ______". Role prompting can also change the style, tone, and manner of Claude's response.

**Note:** Role prompting can happen either in the system prompt or as part of the User message turn.

### Examples

In the example below, we see that without role prompting, Claude provides a **straightforward and non-stylized answer** when asked to give a single sentence perspective on skateboarding.

However, when we prime Claude to inhabit the role of a cat, Claude's perspective changes, and thus **Claude's response tone, style, content adapts to the new role**. 

**Note:** A bonus technique you can use is to **provide Claude context on its intended audience**. Below, we could have tweaked the prompt to also tell Claude whom it should be speaking to. "You are a cat" produces quite a different response than "you are a cat talking to a crowd of skateboarders.

Here is the prompt without role prompting in the system prompt:

In [4]:
system_prompt = ""
user_prompt = "In one sentence, what do you think about skateboarding?" 
raw_prompt, assistant_response = get_completion(
    user_prompt, 
    system_prompt=system_prompt
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange(user_prompt, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
In one sentence, what do you think about skateboarding?<|im_end|>
<|im_start|>assistant
<think>

</think>


```


---
#### Exchange (User/Assistant)
### 🐶 

In one sentence, what do you think about skateboarding? 
### 🤖 
Skateboarding is a dynamic and expressive sport that combines creativity, balance, and street culture.

---

Here is the same user question, except with role prompting.

In [5]:
system_prompt = "You are a cat."
user_prompt = "In one sentence, what do you think about skateboarding?" 
raw_prompt, assistant_response = get_completion(
    user_prompt, 
    system_prompt=system_prompt
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange(user_prompt, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
You are a cat.<|im_end|>
<|im_start|>user
In one sentence, what do you think about skateboarding?<|im_end|>
<|im_start|>assistant
<think>

</think>


```


---
#### Exchange (User/Assistant)
### 🐶 

In one sentence, what do you think about skateboarding? 
### 🤖 
I think skateboarding is a fun and creative way to move around, but I'd rather nap in a sunny spot.

---

You can use role prompting as a way to get Claude to emulate certain styles in writing, speak in a certain voice, or guide the complexity of its answers. **Role prompting can also make Claude better at performing math or logic tasks.**

For example, in the example below, there is a definitive correct answer, which is yes. However, Claude gets it wrong and thinks it lacks information, which it doesn't:

### Note: This is my Solution to the Problem
### Problem 
- Let S be "Married Person looking at Unmarried Person"

#### Case 1: Assumption: Anne is Married
- S1: Jack (Married) -> Anne (Married) = False
- S2: Anne (Married) -> George (Unmarried) = True
- S: S1 or S2 = True
- Hence, S is true if Anne is Married

#### Case 2: Assumption: Anne is Unmarried
- S1: Jack (Married) -> Anne (Unmarried) = True
- S2: Anne (Unmarried) -> George (Unmarried) = False
- S: S1 or S2 = True
- Hence, S remains true if Anne is Unmarried

Therefore:
- By Case 1 and Case 2, the question S always holds. (QED) 


In [6]:
system_prompt = "" 
user_prompt = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"


raw_prompt, assistant_response = get_completion(
    user_prompt, 
    system_prompt=system_prompt
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange(user_prompt, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?<|im_end|>
<|im_start|>assistant
<think>

</think>


```


---
#### Exchange (User/Assistant)
### 🐶 

Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person? 
### 🤖 
Let's break down the information step by step to determine whether a **married person is looking at an unmarried person**.

### Given:
1. **Jack is looking at Anne.**
2. **Anne is looking at George.**
3. **Jack is married.**
4. **George is not married.**
5. **We don’t know if Anne is married.**

---

### Step 1: Identify who is married and who is not
- **Jack is married.**
- **George is not married.**
- **Anne's marital status is unknown.**

---

### Step 2: Determine who is looking at whom
- **Jack is looking at Anne.**
- **Anne is looking at George.**

So:
- **Jack (married) is looking at Anne (unknown).**
- **Anne (unknown) is looking at George (not married).**

---

### Step 3: Analyze the relationships

We are asked: **Is a married person looking at an unmarried person?**

Let’s consider the two possibilities for Anne:

#### Case 1: Anne is married
- Then:
  - Jack (married) is looking at Anne (married).
  - Anne (married) is looking at George (not married).
- In this case, **a married person (Anne) is looking at an unmarried person (George)**.
- So the answer is **yes**.

#### Case 2: Anne is not married
- Then:
  - Jack (married) is looking at Anne (not married).
  - Anne (not married) is looking at George (not married).
- In this case, **a married person (Jack) is looking at an unmarried person (Anne)**.
- So the answer is also **yes**.

---

### Conclusion:

In **both cases**, a **married person is looking at an unmarried person**.

### ✅ Final Answer:
**Yes, a married person is looking at an unmarried person.**

---

### Note:
It seems Qwen already gets this without any role playing.

In [7]:
system_prompt = "You are a logic bot designed to answer complex logic problems."
user_prompt = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"


raw_prompt, assistant_response = get_completion(
    user_prompt, 
    system_prompt=system_prompt
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange(user_prompt, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
You are a logic bot designed to answer complex logic problems.<|im_end|>
<|im_start|>user
Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?<|im_end|>
<|im_start|>assistant
<think>

</think>


```


---
#### Exchange (User/Assistant)
### 🐶 

Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person? 
### 🤖 
Let's break this down step by step using logic.

We are told:

1. **Jack is looking at Anne.**
2. **Anne is looking at George.**
3. **Jack is married.**
4. **George is not married.**
5. **We don’t know if Anne is married.**

We are asked: **Is a married person looking at an unmarried person?**

---

### Step 1: Identify who is married and who is not

- **Jack is married.**
- **George is not married.**
- **Anne's marital status is unknown.**

---

### Step 2: Determine who is looking at whom

- **Jack is looking at Anne.**
- **Anne is looking at George.**

So:

- **Jack (married) is looking at Anne (unknown).**
- **Anne (unknown) is looking at George (not married).**

---

### Step 3: Analyze the relationships

We are looking for a situation where **a married person is looking at an unmarried person**.

Let’s consider the two possibilities:

#### Case 1: Anne is married
- Then:
  - Jack (married) is looking at Anne (married).
  - Anne (married) is looking at George (not married).
- In this case, **a married person (Anne) is looking at an unmarried person (George)**.
- So the answer is **yes**.

#### Case 2: Anne is not married
- Then:
  - Jack (married) is looking at Anne (not married).
  - Anne (not married) is looking at George (not married).
- In this case, **a married person (Jack) is looking at an unmarried person (Anne)**.
- So the answer is also **yes**.

---

### Conclusion:

In **both cases**, a married person is looking at an unmarried person.

### ✅ Final Answer: **Yes**.

---